In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from VirtualWorld.World import World

In [ ]:
world = World()

In [ ]:
world.simulate_time(6)

In [ ]:
world.discussions[37].discussion.discussion_chart_vals()

In [ ]:
# Discussion 1 
opinions = [0.33, 0.55, -0.64, -0.18]
participants = ["Ada Lawson", "Johnnie Helm", "Ashley Thurston","Helga Bass"]

# Discussion 2
# participants = ["Shirley Johnson", "Mary Lindhorst", "James Pease"]
# opinions = [1, -1, 0.34]

# Discussion 3 
# opinions = [0.85, 0.73, -0.11, 0.70, -0.08, -0.01, 0.58, -0.95, 0.93]
# participants = ["Angelo", "Josephine", "Juan", "Karen", "Kathleen", "Kevin", "Lashawna", "Patrice", "Sarah"]

# Discussion 4
# participants = ["Norma Garner", "Kenneth Lyman", "William Mahaffey-Cochran", "Amy Welsh", "Edward Moore"]
# opinions = [0.78, -1, 0.04, -0.21, 1]

op_np = np.zeros((len(opinions), 2))
for index, op in enumerate(opinions): 
    op_np[index, 0] = op

In [ ]:
import numpy as np
from statistics import mean
from collections import defaultdict
import jenkspy
import numpy as np

def find_best_grouping_of_opinions():
    def small_grouping_opinions():
        opinions.sort()
        maximumGap = 0
        split = 0

        for i in range(len(opinions) - 1):
            if(maximumGap < (opinions[i + 1] - opinions[i])):
                maximumGap = opinions[i + 1] - opinions[i]
                split = opinions[i]
        groups = [[num for num in opinions if num <= split], [num for num in opinions if num > split]]
        opinion_grouping = defaultdict(list)
        participant_grouping = defaultdict(list)
        for group in groups:
            if len(group) > 0: 
                mean_group = round(mean(group), 2) 
                opinion_grouping[mean_group] = group
                for index, op in enumerate(opinions): 
                    if op in group: 
                        person = participants[index]
                        participant_grouping[mean_group].append(participants[index])
                        
        return opinion_grouping, participant_grouping

    def get_classified_ops(zone_indices, show_participants=False):
        if not show_participants: 
            array_sort = [np.array([opinions[index] for index in zone]) for zone in zone_indices if zone]
            return array_sort

        else:
            array_sort = [[opinions[index] for index in zone] for zone in zone_indices if zone]
            participants_sort = [[participants[index] for index in zone] for zone in zone_indices if zone]

            opinion_grouping = defaultdict(list)
            participant_grouping = defaultdict(list)
            
            for index, zone in enumerate([zone for zone in array_sort]): 
                mean_op = round(mean(zone), 2)
                opinion_grouping[mean_op] = [op for op in zone]
                participant_grouping[mean_op] = participants_sort[index]

            return [opinion_grouping, participant_grouping]


    def goodness_of_variance_fit(nclasses):

        # Breaks will include the min/max of the data 
        # 3 is the min num_classes that is taken into the breaks --> i.e. produces 2 classes 
        classes = jenkspy.jenks_breaks(opinions, nb_class=nclasses)
        classified = np.array([classify(i, classes) for i in opinions])
        maxz = max(classified)
        zone_indices = [[idx for idx, val in enumerate(classified) if zone + 1 == val] for zone in range(maxz)]

        array_sort = get_classified_ops(zone_indices)
        sdam = np.sum((opinions - np.mean(opinions)) ** 2)
        sdcm = sum([np.sum((classified - classified.mean()) ** 2) for classified in array_sort])
        gvf = (sdam - sdcm) / sdam

        return gvf, zone_indices

    def classify(value, breaks):
        for i in range(1, len(breaks)):
            if value <= breaks[i]:
                return i
        return len(breaks) - 1


    gvf = 0.0
    nclasses = 2

    if len(opinions) <= 3:
        return small_grouping_opinions()
        
    # Goodness of fit set to 0.9
    best_gvf = 0.0
    best_zone_indices = None
    while nclasses < len(opinions) and gvf < 0.9:
        gvf, zone_indices = goodness_of_variance_fit(nclasses)
        if gvf > best_gvf: 
            best_gvf = gvf
            best_zone_indices = zone_indices
            print(best_gvf, get_classified_ops(best_zone_indices, show_participants=True)[1].values())

        nclasses += 1
        
    return get_classified_ops(best_zone_indices, show_participants=True)

In [ ]:
find_best_grouping_of_opinions()

[['Index',
  'Quotes',
  'Sasha',
  'Ameya add',
  'Ameya minus',
  'Chinmaya add',
  'Chinmaya minus'],
 ['16',
  '1-19: Helga started at Left; moved to centrist and then closed at left',
  '#NPCMentionedUnprompted #ChangedOpinion #Believable',
  '#StandingGround',
  '#Believable',
  '#Expected',
  '#StandingGround'],
 ['17',
  '1-20: That the right leaning group, one of whose members was very certain about his opinion, did budge in their convictions ',
  '#InfluenceIndividual #CertaintyConvinces #StandingGround #IdentifyingSimilarGroups #ChangedOpinion #Believable',
  '',
  '#ChangedOpinion #Believable',
  '#Expected #ChangedOpinion',
  '#StandingGround'],
 ['30',
  "2-14: No substantial agreement was reached; which is what you might expect from an argument where people's views start out very highly separated from each other.",
  '#StandingGround #Polarization #CausalInference #Disagreement #Believable',
  '',
  '#Disagreement #Believable',
  '#Expected',
  '#StandingGround'],
 ['43'